In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None

import numpy as np
import matplotlib.pyplot as plt

# load in data
data_path='../data_extraction/data/'

In [150]:
cohort = pd.read_table(data_path+'cohort.tsv')

In [ ]:
# numerical variable summaries
#print(cohort[['age','los','max_lactate','vaso_frac']].describe())

5.0

In [181]:
icustay = 202634 # empty weight
patientweight = cohort.loc[cohort['icustay_id'] == icustay]['weight']
#patientweight.isnan
type(patientweight)
patientweight.values[0]
if np.isnan(patientweight.values[0]):
    print('yes')


yes


In [253]:
# Process urine values 

urine = pd.read_csv(data_path+'urine.csv')

#urine['four_hour_block'] = pd.Series((urine.min_from_intime/interval).astype(int), index=maps.index)

icustays = np.unique(urine['icustay_id'])

# For each icustay key, a list of: [array of urine output volumes in 4h windows, starting time of windows, AKI window indices]  
urine_4h = {}
# Binary outcome of aki or not for each icustay. 
aki_urine ={} #np.zeros(len(icustays)) 

for icustay in icustays:
    # Get all the urine values and times for the icustay_id 
    u= np.array(urine.loc[urine['icustay_id'] == icustay]['value']) 
    t = np.array(urine.loc[urine['icustay_id'] == icustay]['min_from_intime'])
    
    # Keep the time of first urine measurement, and get relative times. 
    t0=t[0]
    t=t-t0
    
    # Calculate urine output in 4 hour blocks starting from the time of first urine measurement. 
    # Hence the first urine measurement will not be used.
    
    # Urine volumes for four hour blocks, starting from the first measurement extending to or before the last urine measurement. 
    # Urine output for the block before the first measurement is not calculated.
    nblocks = int(np.ceil(t[-1]/ 240))
    urine_blocks = np.zeros(nblocks)
    
    # For every urine measurement, add the proportionate volume to the appropriate 4h windows. 
    for ind in range(1, len(u)):
        
        # Which 4h block index the measurement falls in 
        blocknum = int(t[ind]/ 240 )
        if blocknum == nblocks:
            blocknum = blocknum-1
        
        # Left time limit of the rectangle to calculate urine volume proportion that fits into block. 
        leftlimit_t = max(t[ind-1], int(t[ind]/240)*240)
        
        # The proportion of urine volume that belongs to the current 4h block
        propcurrent = (t[ind]-leftlimit_t)/(t[ind]-t[ind-1])
        
        urine_blocks[blocknum] += u[ind]*propcurrent
        
        if (leftlimit_t!=t[ind-1])&(blocknum>0) :
            urine_blocks[blocknum-1] += u[ind]*(1-propcurrent)
                        
    
    # Get the patient weight to calculate RIFLE criteria. 
    patientweight = cohort.loc[cohort['icustay_id'] == icustay]['weight'].values
    if np.isnan(patientweight[0]):
        patientweight=50 # temporary                          
            
    # Whether the urine block meets the I criteria
    urine_blocks_I = urine_blocks < (2 * patientweight)      
            
    # Find 3 consecutive 4h blocks that satisfy the I kidney injury criteria   
    aki_urine[icustay]=0
    for in in range(0, nblocks-2):
        if np.array_equal(urine_blocks_I, [True, True, True]):
            
    
    
    
        
    urine_4h[icustay]=[urine_blocks, t0, ]
    
                

/home/chen/Software/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:43: RuntimeWarning: invalid value encountered in double_scalars
/home/chen/Software/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:59: RuntimeWarning: invalid value encountered in less


In [264]:
a=np.array([10,20,30,40])
b=a<30
print(b)
print(b[0:2])
np.array_equal(b[0:2], [True, False])

[ True  True False False]
[ True  True]


False

In [258]:
x=[10,20,30,20]

for i in range(0, len(x)):
    print(i)
    print(x[i])

0
10
1
20
2
30
3
20


In [254]:
#type(urine_4h)
urine_4h
aki_urine

{286720: 1,
 294915: 0,
 229381: 1,
 245767: 1,
 253961: 1,
 262154: 1,
 213003: 1,
 253964: 1,
 229389: 1,
 204815: 1,
 298970: 1,
 253969: 1,
 229396: 1,
 213013: 1,
 204823: 1,
 213016: 1,
 245785: 1,
 264327: 1,
 243034: 1,
 265691: 1,
 270367: 1,
 283995: 1,
 287241: 1,
 221221: 1,
 276057: 1,
 213031: 1,
 221224: 1,
 270377: 0,
 278571: 1,
 270380: 1,
 221230: 1,
 278575: 0,
 237617: 1,
 204850: 1,
 213043: 1,
 213044: 1,
 262203: 1,
 294607: 1,
 237632: 1,
 237634: 1,
 203446: 1,
 270406: 1,
 245834: 1,
 294989: 1,
 294925: 1,
 262226: 1,
 259427: 1,
 277968: 1,
 221271: 1,
 221273: 1,
 278618: 1,
 278946: 1,
 229469: 1,
 213086: 1,
 254050: 1,
 262244: 1,
 284006: 1,
 237678: 1,
 238952: 1,
 278642: 1,
 262259: 1,
 260798: 1,
 284009: 1,
 262164: 1,
 204924: 1,
 296298: 1,
 229502: 1,
 245781: 1,
 296746: 1,
 204938: 1,
 253975: 1,
 245900: 1,
 204941: 0,
 229521: 1,
 285379: 0,
 273709: 0,
 262169: 1,
 221336: 1,
 262297: 1,
 204954: 1,
 286875: 1,
 255875: 1,
 250965: 1,
 245

In [118]:
#[250, 500, 719]
#[240, 480, 480]

x=720
int(x/240)*240


720

In [6]:
# convert raw MAP readings to MAP features
maps = pd.read_csv(data_path+'map.csv')
interval = 60

# average MAP for every hour
maps['hour'] = pd.Series((maps.min_from_intime/interval).astype(int), index=maps.index)

# get means for every hour with data present. 
mean_maps = maps.groupby(['icustay_id', 'hour'])['value'].mean()
mean_maps = mean_maps.to_frame().reset_index().set_index(['icustay_id'])


# interpolate MAPs for missing hours

# For each icustay, 
min_hours = mean_maps.groupby([mean_maps.index.get_level_values(0)])['hour'].min()
max_hours = mean_maps.groupby([mean_maps.index.get_level_values(0)])['hour'].max()

interp_index = []
for this_icustay in min_hours.index:
    min_hour = min_hours.loc[this_icustay]
    max_hour = max_hours.loc[this_icustay]
    test =[hour for hour in np.arange(min_hour,max_hour+1)]
    interp_index += [(this_icustay, hour) for hour in np.arange(min_hour,max_hour+1)]

mean_maps = mean_maps.set_index(['hour'],append=True)
interp_mean_maps = mean_maps.reindex(pd.MultiIndex.from_tuples(interp_index,names=['icustay_id','hour']))
interp_mean_maps = interp_mean_maps['value'].interpolate(method='linear')
interp_mean_maps = interp_mean_maps.to_frame().reset_index()

# get percent of hours missing a MAP value
missing_map = len(interp_mean_maps.index) - len(mean_maps.index)

frac_missing = missing_map/float(len(interp_mean_maps.index))
print("Fraction of hours missing MAP values:", frac_missing)

# get minimum MAP value per patient
min_ind = interp_mean_maps.groupby('icustay_id')['value'].idxmin(skipna=True)
min_maps = interp_mean_maps.loc[min_ind]

# bin MAP values for first 72 hours
map_72 = interp_mean_maps.loc[interp_mean_maps['hour']<72]

map_cutoffs = np.append(np.arange(30,100,10),200)
map_72['bins'] = pd.cut(map_72['value'], map_cutoffs)
binned_min_maps = pd.cut(min_maps['value'], map_cutoffs)

map_fracs = map_72.groupby('icustay_id')['bins'].value_counts(normalize=True)

('Fraction of hours missing MAP values:', 0.07394823073129662)


In [128]:
urine

array([10, 20, 30, 40, 50])

In [ ]:
maps.head

In [29]:
area=np.trapz([0,1,1], [0, 1.2, 5])
area

4.3999999999999995

In [37]:
t=np.array([0,1,2,3])
u=np.array([0,1,2,1])
dt=np.diff(t)
print(dt)



[1 1 1]


In [39]:
area=np.dot(dt, u[1:])
area

4

In [83]:
a={20003: ['jack', [1,2,3]], 20002:['jill', [9,9]] }

['jack', [1, 2, 3]]